# Vykreslení místa vydání exilových časopisů do mapy

V tomto notebooku budeme pracovat s vybranými daty Bibliografie českého literárního exilu. Ta obsahuje záznamy o knihách a statích s literární tématikou, vydaných a publikovaných v českých exilových nakladatelstvích a časopisech. Nás budou zajímat pouze záznamy o statích, u kterých jsou uvedeny časopis a místo vydání. Ty najdeme v poli `773 $t`.  <br>
Ukážeme si, jak z dat získat a upravit místo vydání záznamu, zjistit jeho souřadnice a následně místo zobrazit na mapě pomocí knihovny `geopandas` a `matplotlib`.   <br>

Tento notebook je určený jak pro začátečníky, tak pro ty, kteří se chtějí seznámit se zpracováním bibliografických dat v Pythonu.

## Požadavky

K práci s notebookem je potřeba mít vytvořený CSV soubor z notebooku Explore Data.<br>

## Předpoklady

Tento notebook nepředpokládá hluboké znalosti Pythonu, ale základní znalost programování bude užitečná.<br> 

## Struktura notebooku

Notebook je rozdělen do několika částí:

0. **Příprava**: Přidáme potřebné knihovny, které budeme používat ke zpracování marcového souboru. 

1. **Načtení z CSV**: Ukážeme si, jak načíst naše data uložená v CSV.

2. **Extrahování místa vydání**: Z dat najdeme místo zjistíme místo vydání.

3. **Čistění dat**: Místa vydání, která obsahují překlepy atp. opravíme. 

4. **Zjištění souřadnic míst**: Naučíme se, jak pomocí API zjistit souřadnice místa vydání. Výsledek si uložíme do CSV souboru.

5. **Načtení souřadnic**: Souřadnice načteme z CSV souboru.

6. **Mapa**: Na závěr naše data zaneseme do mapy, kterou i vykreslíme.

## Další zdroje

- [LearnPython.org](https://www.learnpython.org/): Tento online kurz nabízí výuku jazyka Python pro začátečníky i pokročilé. Může být užitečným zdrojem pro ty, kteří chtějí rozšířit své znalosti Pythonu.
- [W3Schools.com/Python](https://www.w3schools.com/python/): Obsáhlý tutoriál, který provází i některými oblíbenými knihovnamy Pythonu. 




### 0. Příprava 
Jako první si musíme nainstalovat knihovny, se kterými budeme pracovat. Knihovny jsou balíčky funkcí, které nejsou součástí základu jazyka python. <br>
Knihovny nainstalujeme pomocí příkazu `%pip install <jmeno_knihovny>` . Pak je do našeho notebooku přidáme pomocí příkazu `import <jmeno_knihovny> (as alias)`. K funkcím knihovny se pak přistupuje `jmeno_knihovny.jmeno_funkce` <br> 
Pokud z knihovny chceme využít pouze jednu funkci, přidáme ji pomocí `from <jmeno_knihovny> import <jmeno_funkce>`


In [ ]:
%pip install geopandas
%pip install matplotlib
%pip install numpy 
%pip install pandas
%pip install requests
%pip install shapely

from collections import Counter
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
import re
import requests
from shapely.geometry import Point
from statistics import median, mean

### 1. Načtení dat

Nejprve pomocí knihovny pandas načteme naše uložená CSV data do datové struktury DataFrame (která je podobná např. excelovské tabulce). Řádky v DataFramu reprezentují jednotlivé záznamy, sloupce pak jeden typ (např. jmeno autora).
Některá pole a podpole mohou opakovat, ty jsou  v CSV spojené středníkem. V DataFramu pak funkcí `split()` hodnoty rozpojíme a převede do listu (seznamu). Pokud zrovna na daném místě žádná hodnota není, přidáme prázdný list.

In [ ]:
# Vybereme bazi
base = 'cle'

# Cesta k nasim datum
csv_data = 'data/csv/out_{base}.csv'.format(base = base)

# Nacteni dat
df = pd.read_csv(csv_data, delimiter=',')

print("Data načtena do DataFramu df.")

for column in df.columns:
    if column != 'year':   
        # Hodnoty spojene v jeden string zpatky rozdelime do listu, aby se nam s nim lepe pracovalo
        df[column] = df[column].apply(lambda x: x.split(';') if isinstance(x, str)  else [])


Podíváme, jak naše data v tabulce vypadají. Nejprve si vypíšeme, kolik záznamů obsahuje informaci o časopisu. To jsou záznamy o statích, které nás budou zajímat. Zbylé budeme považvat za záznamy o knihách. Pak si vypíšeme prvních 5 a posledních 5 položek v DataFramu. <br>

Nejprve si pomocí lambda funkce najdeme řádky, které obsahují informaci o časopisu. K nim přiřadíme 1 a všechny sečteme. Tím získáme celkový počet záznamů o statích. <br>
Od celkového počtu záznamů pak počet záznamů o statích odečteme, čímž získáme počet záznamů o knihách. Do našeho DataFramu `df` si nakonec uložíme jen záznamy o statích. 

###### Pokud bychom chtěli přesná data, v marcovém záznamu je kolonka LDR (leader), která nese informaci o typu záznamu. 

In [ ]:
# Spocteme radky, ktere v maji nejakou  hodnotu
magazines_counts = df['magazine'].apply(lambda x: 1 if len(x) > 0 else 0)

sum_magazines_counts = magazines_counts.sum()

print("Počet záznamů o statích: ", sum_magazines_counts)

sum_books_counts = len(df) - sum_magazines_counts

print("Počet záznamů o knihách: ", sum_books_counts)

Jak vidíme, statě tvoří většinu databáze.<br>

In [ ]:
# Vypise prvnich 5 zaznamu v DataFramu
df.head()

In [ ]:
# Vypise poslednich 5 zaznamu v DataFramu
df.tail()

Vypsali jsme si i začátek a konec našeho DataFramu. Můžeme si všimnout, že u jména časopisu je v kulatých závorkách napsané místo vydání, a že není napsáno u všech. 

### 2. Extrahování místa vydání

Vyextrahujeme si tedy místo vydání a zjisíme, kolik záznamů nemá místo vydání. K tomu využijeme regulární výraz, který nám najde řetězec (slovo) v kulatých závorkách. Záznamy, které mají časopis, ale ne místo vydání, budou vracet hodnotu None. Hodnoty None pak spočteme. <br>

In [ ]:
# regex patern, ktery najde substring mezi zavorkami
pattern_cities = r"\((.*?)\)"

# Ulozime mesta do promenne jako list
cities = df['magazine'].apply(lambda x: [re.search(pattern_cities, y).group(1) if re.search(pattern_cities, y) else None for y in x]).tolist().copy()

# Spocteme vsechny hodnoty, ktere v sobe maji None hodnotu, a tedy nemaji misto vydani
sum_None =  sum(list(map(lambda x : 1 if (len(x) == 1 and x[0] is None) else 0, cities )))

print("Počet časopisů bez místa vydání: ", sum_None)

Jak vidíme, není to příliš mnoho záznamů, a proto se jimi nemusíme trápit. <br>

Teď už nám jen zbývá zjistit místa vydání a jejich četnost výskytů. <br> 

Abychom nemuseli psát stejný kód několikrát, napíšeme ho jednou do funkce, kterou pak jednoduše zavoláme. V tomto případě si napíšeme funkci, která nám z několika listů (seznamů) vnořených do sebe vytvoří jeden. To se nám bude hodit, až budeme chtít spočítat četnost výskytů míst vydání. 

In [ ]:
# Funkce na "zplosteni" listu -> ze sloupce DataFramu vytvori jeden list se vsemi elementy 
def flatten_list(strings):
    flattened_list = []
    if strings is not None: # Zkontrolujeme, jestli element neni None
        for item in strings:
            if isinstance(item, str):  # Pokud je element string, pridame ho do seznamu
                flattened_list.append(item)
            elif isinstance(item, list):  # Rekurzivne zavola sebe
                flattened_list.extend(flatten_list(item))
        return flattened_list

print("Funkce uložena")         

Zjistíme, jaká místa vydání se v naší databázi objevují.

In [ ]:
# Vytvorime zplostely list, ktery i odstrani None hodnoty
cities = flatten_list(cities)

print("Unikátní hodnoty:", np.unique(cities))

### 3. Čistění dat

Z uniktních hodnot vidíme, že některá města jsou uložena jak pod svým původím jménem, tak pod svou českou alternativou (London - Londýn, Köln - Kolín nad Rýnem). U některých jsou překlepy (Wintertuhr-Obstladen -> Winterthur-Obstalden), které musíme opravit. U některých je zase měst více (Wintertuhr-Obstladen , Ženeva-Middlesex-Mnichov). V tomto případě si všechna města uložíme v listu.<br>
Použijeme k tomu funkci `lambda` a `map()`, která funguje stejně jako funkce `apply()` v knihovně `pandas`.


In [ ]:
# Odstranime Index on Censorship z elementu "London, Index on Censorship" a spojime "London" a "Londýn"
cities = list(map(lambda x: 'Londýn' if 'London' in x else x, cities))

# Odstranime 'Obstladen' z Winterthur
cities = list(map(lambda x: ['Winterthur', 'Obstalden'] if 'Obstladen' in x else x, cities))

# Vytvorime dva elementy z New York-Paříž
cities = list(map(lambda x: ['New York','Paříž'] if 'New York-Paříž' in x else x, cities))

# Vytvorime tri elementy z Ženeva-Middlesex-Mnichov
cities = list(map(lambda x: ['Ženeva','Middlesex', 'Mnichov'] if 'Ženeva-Middlesex-Mnichov' in x else x, cities))

# Prepiseme Köln-Ehrenfeld na Kolín nad Rýnem 
cities = list(map(lambda x: 'Kolín nad Rýnem' if 'Köln-Ehrenfeld' in x else x, cities))

# Zplostime seznam
cities = flatten_list(cities)

print("Unikátní hodnoty po změně:", np.unique(cities))

Pomocí funkce `Counter()` spočteme četnost míst vydání. 

In [ ]:
# Spocteme vyskyty mest
cities_number_of_records = Counter(cities)

print(cities_number_of_records)

Převedeme z dictionary do DataFramu `cities_df`. 

In [ ]:
# Vytvorime DataFrame 
cities_df = pd.DataFrame.from_dict(cities_number_of_records, orient='index').reset_index()

# Popiseme sloupce
cities_df.columns = ['city', 'number of records']

# Vypiseme 
cities_df

### 4. Zjištění souřadnic míst

Tento kód je ukázka toho, jak pomocí API získat souřadnice měst. Je k němu potřeba osobní API klíč, který lze zíkat z této stránky - https://opencagedata.com/api. Ten se jen přidá do proměnné `api_key`. Kód pak vyšle request přes url stránku s naším API klíčem a jménem města. Pokud request proběhne v pořádku, funkce vrátí zeměpisnou šířku (latitude) a výšku (longitude)<br>

<i> Tato buňka se nespustí. Pro spuštění je potřeba vymazat první řádek `%%script echo skip` </i>


In [ ]:
%%script echo skip

# funkce, ktera podle jmena mesta najde jeho souradnice
def get_city_coordinates(city):
    api_key = "MY KEY"
    url = f"https://api.opencagedata.com/geocode/v1/json?q={city}&key={api_key}"
    
    # Volani funkce requests
    response = requests.get(url)

    data = response.json()
    
    if response.status_code == 200:
        # pokud se mesto podarilo najit, ulozime souradnice
        if data["total_results"] > 0:
            lat = data["results"][0]["geometry"]["lat"]
            lon = data["results"][0]["geometry"]["lng"]
            return lat, lon
        else:
            print("No results found for the city.")
    else:
        print("Error occurred while fetching data.")

Pomocí funkce get_city_coordinates, kterou jsme si napsali v předchozí buňce, zjistíme souřadnice měst, která si zapíšeme do DataFramu. Na konci se DataFrame uloží do CSV souboru. 

<i> Tato buňka se nespustí. Pro spuštění je potřeba vymazat první řádek `%%script echo skip` </i>


In [ ]:
%%script echo skip

cities_df['latitude'] = None
cities_df['longitude'] = None
df.reindex(columns=['city', 'number of records', 'lat', 'lon'], fill_value=0)

# Unikatni mesta 
unique_cities = np.unique(cities)
coordinates = {}

# Iterace pres vsechny unikatni mesta
for city in unique_cities:
    # Pouzijeme funkci try-except, ktera nam odchyti pripadne chyby
    try:
        (latitude, longitude) = get_city_coordinates(city)
        print(f"Coordinates of {city}: Latitude={latitude}, Longitude={longitude}")
        coordinates[city] = (latitude, longitude)
        cities_df.loc[cities_df['city'] == city, 'latitude'] = latitude
        cities_df.loc[cities_df['city'] == city, 'longitude'] = longitude
    except:
        print(f"City {city} not found.")
          
# Data ze slovniku dame do DataFramu
df_coordinates = pd.DataFrame.from_dict(coordinates)

# Tabulku transponujeme 
df_coordinates = df_coordinates.T

df_coordinates.to_csv('data/coordinates/coordinates.csv')

### 5. Načtení souřadnic

Výsledek přechozícho kódu jsme si uložili do souboru coordinates.csv, ze kterého si data teď načteme.   
CSV soubor obsahuje seznam míst, u kterých se připsaná zeměpisná šířka (latitude) a výška (longitude).

In [ ]:
# Nacteme souradnice ze souboru
df_coordinates = pd.read_csv('data/coordinates/coordinates.csv')

# Prejmenujeme sloupce
df_coordinates.columns = ['city','latitude', 'longitude']

# Spojime tabulky dohromady
points_df = pd.merge(cities_df, df_coordinates)

points_df

### 6. Mapa

V poslední části si data vykreslíme do mapy, kterou máme připravenou ve složce data/geojson. Nejprve si určíme, jakou část světa budeme chtít vykreslit. Pak postupně města-body zaneseme do mapy. Velikost bodu se bude odvíjet od počtu vydaných článků. Přidáme i legendu, případně i popisky k jednotlivým městům.  

#### 6.1 Ohraničení mapy

Jelikož se všechna města nachází buď v Evropě nebo v severní Americe, můžeme mapu světa, kterou budeme vykrelovat, ohraničit. Ohraničení definujeme pomocí minimální a maximální zeměpisné výšky a šířky. Následně pak vyřadíme všechna města, která jsou mimo ohraničení. <br>


In [ ]:
# Hranice, kterou chceme ohranicit 
bbox = [-130, 15, 50, 80]  # [minx, miny, maxx, maxy] - minimalni longitude, minimalni latitude, maximalni longitude, maximalni latitude 

# Seznam mest, ktera chceme vyradit 
remove = []

# Iterace skrz DataFrame
for _, row in points_df.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    minx = bbox[0]
    miny = bbox[1]
    maxx = bbox[2]
    maxy = bbox[3]
    
    # Vsechny mesta, jejich zem. sirka nebo vyska lezi mimo box 
    # si pridame do seznamu remove
    if lon < minx or lat < miny or lon > maxx or lat > maxy:
        remove.append(row['city'])

# Vyradime mesta ktera jsou v seznamu remove
points_df = points_df[~points_df['city'].isin(remove)]

points_df

<div class='alert alert-block alert-info'>
    <b>Try It!</b>  Můžeme si vyskoušet nastavení různé zeměpisné šířky a výšky. <br>
Např.  [-130, 15, 50, 80] je hranice severní Ameriky a Evropy. [-5, 35, 30, 55] je přibližné ohraničení Evropy. [6, 45, 11, 48] je přibližně Švýcarská hranice. <br> 
</div>

#### 6.2 Vykreslení do mapy

Pro vykreslení použijeme knihovnu `geopandas` a `matplotlib`.<br> 
Nejprve ze zeměpisné šířky a výšky vytvoříme `Point` z knihovny `shapely` a převedeme do formátu `GeoDataFrame`, který lze jednoduše vykreslit do mapy.<br>
Načteme mapu a vykreslíme. Naše mapa se zobrazí podle ohraničení, které jsme si definovali v předchozí buňce. <br>
Na mapu pak přidáme body - naše místa vydání, podle jejich zeměpisné šírky a výšky. Velikost bodu se bude odvíjet podle počtu vydaných článků na daném místě. <br>

In [ ]:
# Ze zemepisne sirky a vysky vytvorime body 
geometry = [Point(lon, lat) for lon, lat in zip(points_df['longitude'], points_df['latitude'])]

# Prevedeme DataFrame na GeoDataFrame
points_gdf = gpd.GeoDataFrame(points_df, geometry=geometry, crs="EPSG:4326") 

# Nacteni mapy 
base_map_data = gpd.read_file("data/geojson/world_1960.geojson")

# Nastaveni velikosti vykresleni
figsize = (15,12)

plt.figure(figsize=figsize)

# Ohraniceni mapy podle europe_and_north_america_bbox
ax = base_map_data.clip(bbox).plot(figsize=figsize)

# Parametr pro upravovani velikosti bodu
div = 10

# Parametr pro vypsani jmen mest do mapy
ann = False

# Vykresleni mist vydani
points_gdf.plot(figsize=figsize,
                ax=ax, 
                color = "red",
                marker = 'o',
                markersize=points_gdf['number of records'].apply(lambda x: x/div),  # Upraveni velikosti bodu 
                )

# Pokud nase promenna ann je nastavena na True, do mapy vypiseme i jmena mest 
if ann:
     # Pridame vypsani jmen mest do mapy
     for x, y, label in zip(points_df.longitude, points_df.latitude, points_df.city):
          ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points", fontsize = 8)

# Tri moznosti legendy
point_sizes = [min(points_gdf['number of records'].apply(lambda x: int(round(x, -1)))), mean(points_gdf['number of records'].apply(lambda x: int(round(x, -1)))), max(points_gdf['number of records'].apply(lambda x: int(round(x, -1))))]

# Vytvoreni bodu pro legendu
legend_handles = [Line2D([], [], marker = 'o', lw=0, color='red', markersize=np.sqrt(size/div), label=str(int(round(size, -1)))) for size in point_sizes]

# Vykresleni legendy
ax.legend(handles=legend_handles, title='Počet záznamů', loc='upper right')

plt.title("Vydávání exilových časopisů")
plt.grid(False)
ax.set_axis_off()  
plt.savefig("plots/exil_cropped_map.svg")
plt.show()

<div class='alert alert-block alert-info'>
    <b>Try It!</b> Do kódu jsou přidané dva parametry - div a ann.<br> 
Pomocí parametru div můžeme změnit velikost bodu. Čím vyšší číslo, tím menší se bod vykreslí. Vyšší čísla jsou tedy vhodnější pro mapy, které ohraničují větší území, a body se tak nepřekrývají.      
Parametr ann nám umožňuje přidat popisy měst. Hodnota True jméno města přidá k jeho bodu.  
</div>

  